# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chokurdakh,78.002472,146.193437,273.45,90,36,5.14,ru,1691465838
1,1,bluff,-77.093638,171.698325,235.44,100,97,4.21,nz,1691465838
2,2,port alfred,-43.985768,29.357910,283.24,82,91,16.53,za,1691465839
3,3,punta arenas,-78.429197,-88.086156,231.72,96,75,3.94,cl,1691465839
4,4,tumannyy,73.286565,37.833604,281.26,93,100,7.80,ru,1691465840


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria
ideal_weather = city_data_df.loc[(city_data_df["Wind Speed"] <= 4.5) & (city_data_df["Humidity"] <= 50), :]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,79,avarua,-27.248017,-63.524975,286.48,33,94,3.51,ar,1691465871
91,91,paita,46.064382,69.669716,299.13,43,63,3.27,kz,1691465876
101,101,husavik,24.471606,33.087045,305.56,14,0,1.59,eg,1691465879
106,106,kiama,47.224821,53.147573,302.25,33,0,3.94,kz,1691465880
110,110,tsogni,57.823264,136.551931,295.71,32,13,2.45,ru,1691465882
127,127,teguise,-19.671044,18.469717,283.29,47,0,2.12,na,1691465888
207,207,constitucion,-10.147056,-45.912048,296.38,40,5,2.79,br,1691465915
221,221,padang,34.326083,79.818116,278.16,39,9,1.67,in,1691465919
245,245,bathsheba,-19.064831,-53.494962,296.70,33,29,0.77,br,1691465927
265,265,nipawin,-33.207031,149.847459,288.82,42,37,0.92,au,1691465935


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather.drop(columns = ["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
79,avarua,-27.248017,-63.524975,33,ar,
91,paita,46.064382,69.669716,43,kz,
101,husavik,24.471606,33.087045,14,eg,
106,kiama,47.224821,53.147573,33,kz,
110,tsogni,57.823264,136.551931,32,ru,
127,teguise,-19.671044,18.469717,47,na,
207,constitucion,-10.147056,-45.912048,40,br,
221,padang,34.326083,79.818116,39,in,
245,bathsheba,-19.064831,-53.494962,33,br,
265,nipawin,-33.207031,149.847459,42,au,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

SyntaxError: invalid syntax (788381735.py, line 2)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE